In [1]:
import readdata
from text_vectorizer import CV
from text_vectorizer import TFIDF
from text_vectorizer import word2vec
from text_vectorizer import outlierDection
from OutlierDetectRemove import removeOutliers
from sklearn import metrics
from sklearn.svm import SVC
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import GridSearchCV
import numpy as np
import sys

# Usage: 
# 1. python svm.py cv
# 2. python svm.py tfidf
# 3. python svm.py word2vec

def getRemovedVals(X,Y = None,Ftype = "",isTest = False):

    X = np.array(X)
    index,_ = outlierDection(X,Ftype)
    if not isTest:
        Y = np.array(Y)
        Xrem,Yrem = removeOutliers(index,X,Y,Ftype)
        return Xrem,Yrem

    else:
        Xrem = removeOutliers(index,X,Y,Ftype)
        return Xrem

def evaluate(pred, truth):
    print('Mean Absolute Error:', metrics.mean_absolute_error(truth, pred))
    print('Mean Squared Error:', metrics.mean_squared_error(truth, pred))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(truth, pred)))



dfTrain = readdata.read_clean_data(readdata.TRAINFILEPATH,nolabel = False)
dfTest = readdata.read_clean_data(readdata.TESTFILEPATH,nolabel = True)

X_train = dfTrain['text'].to_numpy()
Y_train = dfTrain['label'].to_numpy()
X_test = dfTest['text'].to_numpy() # unlabelled

# X_train, X_test, _ = CV(X_train, X_test) # train shape: (17973, 141221)
# X_train,Y_train = getRemovedVals(X = X_train,Y = Y_train,Ftype = "CV_Train",isTest = False)
# X_test = getRemovedVals(X = X_test,Y = None,Ftype = "CV_Test",isTest = True)


X_train, X_test, _ = TFIDF(X_train, X_test) # shape: (17973, 141221)
X_train,Y_train = getRemovedVals(X = X_train,Y = Y_train,Ftype = "TFIDF_Train",isTest = False)
X_test = getRemovedVals(X = X_test,Y = None,Ftype = "TFIDF_Test",isTest = True)


# X_train, X_test = word2vec(X_train, X_test)
# X_train,Y_train = getRemovedVals(X = X_train,Y = Y_train,Ftype = "W2V_Train",isTest = False)
# X_test = getRemovedVals(X = X_test,Y = None,Ftype = "W2V_Test",isTest = True)

print("Performing Grid Search on SVM...")
svm = SVC()
parameters = {'kernel':('linear', 'rbf'), 'C':(0.25,0.5,0.75)}
grid = GridSearchCV(estimator = svm, param_grid = parameters,n_jobs=-1, cv=3,verbose=1)
grid_result = grid.fit(X_train, Y_train)
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


/Users/KevinWang/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:237: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)
/Users/KevinWang/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
/Users/KevinWang/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:415: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


(1798,)
(17973,)
the percentage of outliers in  TFIDF_Train  is:  10.003894730985367 %
(16175, 10000)  is shape of X for  TFIDF_Train after removing outliers
(16175,)  is shape of Y for  TFIDF_Train  after removing outlier


/Users/KevinWang/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:237: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)
/Users/KevinWang/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
/Users/KevinWang/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:415: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


(450,)
(4494,)
the percentage of outliers in  TFIDF_Test  is:  10.013351134846461 %
(4044, 10000)  is shape of X for  TFIDF_Test after removing outliers
Performing Grid Search on SVM...
Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed: 138.2min finished


0.940958 (0.002722) with: {'C': 0.25, 'kernel': 'linear'}
0.573849 (0.000050) with: {'C': 0.25, 'kernel': 'rbf'}
0.945100 (0.002506) with: {'C': 0.5, 'kernel': 'linear'}
0.573849 (0.000050) with: {'C': 0.5, 'kernel': 'rbf'}
0.945781 (0.001706) with: {'C': 0.75, 'kernel': 'linear'}
0.573849 (0.000050) with: {'C': 0.75, 'kernel': 'rbf'}


In [2]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.945781 using {'C': 0.75, 'kernel': 'linear'}
